In [4]:
!pip install --upgrade tensorflow tensorflow-datasets

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TextVectorization
import tensorflow_datasets as tfds


print(tf.__version__)

2.19.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_df = "train-data.tsv"
test_df= "valid-data.tsv"

--2025-08-11 09:37:03--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2025-08-11 09:37:03 (9.34 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-08-11 09:37:03--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [10]:
# Clean empty strings
train_df = train_df[train_df['text'].str.strip().astype(bool)]
test_df = test_df[test_df['text'].str.strip().astype(bool)]

train_texts = train_df['text'].values
train_labels = train_df['label'].values.astype(int)
test_texts = test_df['text'].values
test_labels = test_df['label'].values.astype(int)

vectorizer = TextVectorization(max_tokens=max_vocab_size, output_mode='int', output_sequence_length=max_sequence_length)
vectorizer.adapt(train_texts)

model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(input_dim=max_vocab_size, output_dim=64),  # removed mask_zero=True
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_texts, train_labels, epochs=10, validation_data=(test_texts, test_labels))


Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8570 - loss: 0.4215 - val_accuracy: 0.8657 - val_loss: 0.3639
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8684 - loss: 0.3567 - val_accuracy: 0.8657 - val_loss: 0.3326
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8698 - loss: 0.3002 - val_accuracy: 0.9454 - val_loss: 0.1860
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9490 - loss: 0.1454 - val_accuracy: 0.9634 - val_loss: 0.1055
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9799 - loss: 0.0710 - val_accuracy: 0.9641 - val_loss: 0.1002
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9767 - loss: 0.0821 - val_accuracy: 0.9763 - val_loss: 0.0930
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9859 - loss: 0.0514 - val_accuracy: 0.9684 - val_loss: 0.1003
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9870 - loss: 0.0432 - val_accu

In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    inputs = tf.constant([pred_text], dtype=tf.string)  # ensure dtype is string
    spam_prob = float(model.predict(inputs)[0][0])
    label = "spam" if spam_prob > 0.5 else "ham"
    return [spam_prob, label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
[0.012898913584649563, 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
You passed the challenge. Great job!
